In [8]:
import requests
from SPARQLWrapper import SPARQLWrapper, JSON

In [9]:
print("On what topic do you want questions?")
topic = input()
print("How many questions do you want?")
num_questions = int(input())

On what topic do you want questions?
How many questions do you want?


In [10]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

In [11]:
def find_uri_by_label(label):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    
    # SPARQL query to find the URI for a given label
    query = """
    SELECT ?item WHERE {
      ?item rdfs:label "%s"@en.
      SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 1
    """ % label.replace('"', '\"')  # Basic string sanitization
    
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    try:
        results = sparql.query().convert()
        for result in results["results"]["bindings"]:
            return result["item"]["value"]  # Return the first matching URI
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [19]:
def find_Q_by_uri(uri):
    return uri.split("/")[-1]

In [20]:
def find_Q_by_label(label):
    return find_Q_by_uri(find_uri_by_label(label))

In [31]:
def sparql_question(topicQ, predicateQ):
    # Define the SPARQL query
    sparql_query = """
    SELECT ?thing WHERE {
      
    ?thing""" + " wdt:" + predicateQ + " wd:" + topicQ + """.

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    """
    print("running query: " + sparql_query)

    # Set the query and the return format (JSON)
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # Perform the query and convert the result to a Python dictionary
    results = sparql.query().convert()
    return results

In [21]:
find_Q_by_label(topic)

'Q3624078'

In [32]:
sparql_question(find_Q_by_label(topic), find_Q_by_label("instance of"))

running query: 
    SELECT ?thing WHERE {
      
    ?thing wdt:P31 wd:Q3624078.

    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
    }
    LIMIT 10
    


{'head': {'vars': ['thing']},
 'results': {'bindings': [{'thing': {'type': 'uri',
     'value': 'http://www.wikidata.org/entity/Q16'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q17'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q20'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q27'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q28'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q29'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q30'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q31'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q32'}},
   {'thing': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q33'}}]}}

In [ ]:

def get_wikidata_item(item_id):
    # Wikidata API endpoint
    url = "https://www.wikidata.org/w/api.php"
    
    # Parameters for the API request
    params = {
        "action": "wbgetentities",  # Action to get data about entities
        "ids": item_id,  # ID of the Wikidata item (e.g., Q64 for Berlin)
        "format": "json",  # Response format
        "props": "labels|descriptions|aliases",  # Properties to retrieve: labels, descriptions, and aliases
        "languages": "en"  # Language filter
    }
    
    # Make the GET request to the Wikidata API
    response = requests.get(url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        
        # Access the item's data
        item_data = data['entities'][item_id]
        
        # Extract and print the label, description, and aliases
        label = item_data['labels']['en']['value']
        description = item_data['descriptions']['en']['value']
        aliases = [alias['value'] for alias in item_data['aliases']['en']]
        
        print(f"Label: {label}")
        print(f"Description: {description}")
        print(f"Aliases: {', '.join(aliases)}")
    else:
        print("Failed to retrieve data")

# Example usage
get_wikidata_item("Q64")

Label: Berlin
Description: federated state, capital and largest city of Germany
Aliases: Berlin, Germany, Berlin (Germany), DE-BE
